In [1]:
import torch
torch.manual_seed(0)
dropout = torch.nn.Dropout(p=0.5)

X = torch.ones(size=(3, 10), requires_grad=True)
output = dropout(X)
output.retain_grad()

print("output:", output)
print("X:", X)
output.backward(gradient=torch.ones_like(output))
print("output grad after backprop:", output.grad)
print("X grad after backprop:", X.grad)

output: tensor([[2., 0., 0., 2., 2., 0., 0., 2., 2., 2.],
        [2., 0., 0., 2., 2., 2., 0., 2., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 0., 0.]], grad_fn=<MulBackward0>)
X: tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)
output grad after backprop: tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])
X grad after backprop: tensor([[2., 0., 0., 2., 2., 0., 0., 2., 2., 2.],
        [2., 0., 0., 2., 2., 2., 0., 2., 0., 0.],
        [0., 0., 2., 2., 2., 2., 2., 0., 0., 0.]])


/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/torch/autograd/__init__.py:173: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664405705473/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [2]:
from jax import numpy as jnp
import numpy as np
import brunoflow as bf

np.random.seed(0)

dropout_bf = bf.net.Dropout(p=0.5)
dropout_bf.training = True
X = bf.Node(jnp.ones(shape=(3, 10)), name="x")

output = dropout_bf(X)
output.zero_gradients()

print("output:", output)
print("X:", X)
output.backprop()
print("output grad after backprop:", output)
print("X grad after backprop:", X) # note that the grad is only non-zero for the values that were not dropped out (matches value of output)


2022-12-20 15:40:25.507374: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


output: node(name: (* (* x [[1 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 1 1 1]
 [1 1 0 1 0 1 0 1 1 0]]) 2.0), val: [[2. 2. 2. 2. 0. 2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 0. 0. 0. 2. 2. 2.]
 [2. 2. 0. 2. 0. 2. 0. 2. 2. 0.]], grad: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]])
X: node(name: x, val: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]], grad: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]])
output grad after backprop: node(name: (* (* x [[1 1 1 1 0 1 0 1 1 0]
 [1 1 1 1 0 0 0 1 1 1]
 [1 1 0 1 0 1 0 1 1 0]]) 2.0), val: [[2. 2. 2. 2. 0. 2. 0. 2. 2. 0.]
 [2. 2. 2. 2. 0. 0. 0. 2. 2. 2.]
 [2. 2. 0. 2. 0. 2. 0. 2. 2. 0.]], grad: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]])
X grad after backprop: node(name: x, val: [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 